In [1]:
import random

# Simple Tabulation Hash

In [2]:
def particionar(x):
    binario_x = bin(x).zfill(64).replace('b','')

    particoes = []
    for _ in range(8):                   #Aqui utilizo c=8.
        particoes.append(binario_x[-8:]) #Pego os 8 ultimos e adiciono a partição.
        binario_x = binario_x[:-8]       #Atualizo o binario_x retirando os 8 últimos pra continuar a iteração até que não
                                         #sobrem mais caracteres.
    return particoes[::-1]               #Inverto a lista para que os últimos caracteres da repres. binária vão para as
                                         #as últimas partições.

In [3]:
def tabela_aleatoria(): #Aqui teremos 8 tabelas totalmente aleatórias que armazenam 2**c=256 elementos.
    T = []
    random.seed(42)
    for __ in range(8):
        T_i = []
        for _ in range(256):
            valor_aleatorio = random.randint(0,256)
            T_i.append(valor_aleatorio)
        T.append(T_i)
    return T

In [4]:
def funcao_hash(x): #h(x) = T[1](x[1])^T[2](x[2])^...^T[8]x[8]
    particoes = particionar(int(x))
    particao_decimal = [int(particao, 2) for particao in particoes]
    
    T = tabela_aleatoria()
    T_j = []
    for i in range(8):
        j = particao_decimal[i]
        T_j.append(T[i][j])
    
    h_x = T_j[0]^T_j[1]
    for i in range(2,8):
        h_x = h_x^T_j[i]
        
    return h_x

# Funções Auxiliares

In [5]:
def inicializar_tabela(m):
    global contador_elementos
    global contador_flags
    contador_flags = 0
    contador_elementos = 0
    return [None]*m

In [6]:
def limpar(tabela):
    global contador_flags
    global log_operacoes
    
    tabela_antiga = tabela.copy()
    cont_flag = contador_flags
    tabela = inicializar_tabela(len(tabela))
    print("LIMPAR:{}\n\n".format(cont_flag))
    log_operacoes.append("LIMPAR:{}\n\n".format(cont_flag))
    tabela = remapear_valores(tabela_antiga,tabela)
    
    return tabela

In [7]:
def remapear_valores(tabela_antiga, tabela):
    for pos, valor in enumerate(tabela_antiga):
        if(valor != None and valor != '#'): # '#' indica que foi removido um elemento ali
            tabela = incluir(tabela, valor, False)
            
    return tabela

In [8]:
def table_doubling(tabela):
    #global contador_elementos
    global log_operacoes
    #contador_elementos=0
    
    tabela_antiga = tabela.copy()
    m = len(tabela)
    m *= 2
    tabela = inicializar_tabela(m)
    tabela = remapear_valores(tabela_antiga, tabela)
    #print("DOBRAR TAM:{}\n\n".format(len(tabela)))
    #log_operacoes.append("DOBRAR TAM:{}\n\n".format(len(tabela)))
    
    return tabela

In [9]:
def halving(tabela):
    global log_operacoes
    eps=1
    
    if(len(tabela)/2>=(eps+1)): #Aqui restringe a tabela a ter pelo menos tamanho 2.
        tabela_antiga = tabela.copy()
        m = len(tabela)
        m = int(m/2)
        tabela = inicializar_tabela(m)
        tabela = remapear_valores(tabela_antiga, tabela)
        print("METADE TAM:{}\n\n".format(len(tabela)))
        log_operacoes.append("METADE TAM:{}\n\n".format(len(tabela)))

    return tabela

# Operações Hash

In [16]:
def incluir(tabela_hash, x, registrar_log=True):
    global log_operacoes
    global contador_elementos
    global contador_flags
    
    h_x = funcao_hash(x)
    inserido = False
    i=0
    while(not inserido):
        if(i<len(tabela_hash)):            #pra não percorrer a tabela infinitas vezes.
            pos = (h_x+i)%len(tabela_hash) #(h[x]+i)mod m
            if(tabela_hash[pos] == None or tabela_hash[pos] == '#'): # '#' indica que foi removido um elemento ali
                if(tabela_hash[pos] == '#'):
                    contador_flags-=1
                tabela_hash[pos] = x
                inserido = True
                if(registrar_log):
                    print("INC:{}\n{} {}\n\n".format(x, h_x%len(tabela_hash), pos))
                    log_operacoes.append("INC:{}\n{} {}\n\n".format(x, h_x%len(tabela_hash), pos))
            else:
                i+=1
        else:
            tabela_hash = table_doubling(tabela_hash) #se a tabela ja estiver cheia, table doubling remapeando valores.
            while(not inserido):
                if(i<len(tabela_hash)):            #pra não percorrer a tabela infinitas vezes.
                    pos = (h_x+i)%len(tabela_hash) #(h[x]+i)mod m
                    if(tabela_hash[pos] == None or tabela_hash[pos] == '#'): # '#' indica que foi removido um elemento ali
                        if(tabela_hash[pos] == '#'):
                            contador_flags-=1
                        tabela_hash[pos] = x
                        inserido = True
                        if(registrar_log):
                            print("INC:{}\n{} {}\n\n".format(x, h_x%len(tabela_hash), pos))
                            log_operacoes.append("INC:{}\n{} {}\n\n".format(x, h_x%len(tabela_hash), pos))
                            ###
                            print("DOBRAR TAM:{}\n\n".format(len(tabela_hash)))
                            log_operacoes.append("DOBRAR TAM:{}\n\n".format(len(tabela_hash)))
                            
                    else:
                        i+=1
                        
    contador_elementos+=1
    
    return tabela_hash

In [17]:
def buscar(tabela, x, registrar_log=True):
    h_x = funcao_hash(x)
    encontrou = False
    i=0
    
    while(not encontrou):
        if(i<len(tabela)):            #Pra não percorrer a tabela infinitas vezes.
            pos = (h_x+i)%len(tabela) #(h[x]+i)mod m
            if(tabela[pos] == x):          #Caso eu encontre o elemento, retorno sua posição.
                if(registrar_log):
                    print("BUS:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
                    log_operacoes.append("BUS:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
                return pos
            else:
                i+=1
        else:
            encontrou = True
    
    if(registrar_log):
        print("BUS:{}\n{} {}\n\n".format(x, h_x%len(tabela), -1))
        log_operacoes.append("BUS:{}\n{} {}\n\n".format(x, h_x%len(tabela), -1))
    
    return -1                              #Caso o elemento não for encontrado, retorno -1.

In [22]:
def remover(tabela, x): #Implementar o halving (done)
    global contador_elementos
    global contador_flags
    global log_operacoes
    
    h_x = funcao_hash(x)
    pos = buscar(tabela, x, False) 
    if(pos != -1):
        tabela[pos] = '#'
        contador_elementos-=1
        contador_flags+=1
            
        print("REM:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
        log_operacoes.append("REM:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
        
        if(contador_flags>=(len(tabela))*(1/4)):
            tabela = limpar(tabela)
        
        #Fazer o halving quando só 1/4 da tabela estiver preenchida e tabela com tamanho pelo menos (1+epsilon)n.
        if(contador_elementos<=int((1/4)*len(tabela))):
            tabela = halving(tabela)
    else:
        print("REM:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
        log_operacoes.append("REM:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
        
    return tabela

# Execução

In [23]:
contador_elementos = 0
contador_flags = 0
log_operacoes = []

In [24]:
def executar():
    global log_operacoes
    
    tabela_hash = inicializar_tabela(4)
    arquivo = open('entrada.txt','r')
    saida = open('saida.txt','w')
    
    log_operacoes.append("TAM:{}\n\n".format(len(tabela_hash)))
    for line in arquivo:
        operacao, x = line.split(':')
        if(operacao == 'INC'):
            tabela_hash=incluir(tabela_hash, int(x))
            print(tabela_hash)

        elif(operacao == 'REM'):
            tabela_hash=remover(tabela_hash, int(x))
            print(tabela_hash)

        elif(operacao == 'BUS'):
            buscar(tabela_hash, int(x))
            print(tabela_hash)
    
    saida.writelines(log_operacoes)
    
    arquivo.close()
    saida.close()

In [25]:
executar()

INC:10
0 0


[10, None, None, None]
INC:20
1 1


[10, 20, None, None]
REM:10
0 0


LIMPAR:1


METADE TAM:2


[None, 20]
INC:15
1 0


[15, 20]
INC:15
1 3


DOBRAR TAM:4


[None, 15, 20, 15]
REM:17
3 -1


[None, 15, 20, 15]
BUS:42
3 -1


[None, 15, 20, 15]
INC:42
3 0


[42, 15, 20, 15]
INC:43
3 0


DOBRAR TAM:8


[43, 20, None, None, None, 15, 15, 42]
